In [1]:
import numpy as np
import math
import random


In [2]:
#число бросков
L = 300

#матрицы переходов и эмиссий
w = float(1 / 6)
a = np.array([[0.95, 0.05], [0.1, 0.9]], float)
e = np.array([[w, w, w, w, w, w], [0.1, 0.1, 0.1, 0.1, 0.1, 0.5]], float)

#начальное статическое распределение: px - четная кость, py - нечестная, a - матрица распределения
#qA = q
#(x, y)(a) = (x, y)
#x + y = 1
#получаем систему уравнений:
#0.5x + 0.05y = x
#x + y = 1
#P(x, y) = np.array([2/3], [1/3])

In [3]:
#случайная последовательность при заданных параметрах E и A
def HMM(e, a, L):
    if 0 <= random.random() < (2 / 3):
        D = 0
    else:
        D = 1
    path = []
    s = []
    for i in range(L):
        X = random.random()
        K = random.random()
        if (D == 0) and (X < 0.05):
            D = 1
        elif (D == 1) and (X < 0.1):
            D = 0
        if D == 0:
            s.append("0")
            path.append(random.randint(1, 6))
        elif D == 1:
            s.append("1")
            if K < 0.5:
                path.append(6)
            else:
                path.append(random.randint(1, 5))
    return(path, s)


path, s = HMM(e, a, L)

#наблюдение:
print(path)

#состояние:
print(s)

[1, 6, 5, 4, 6, 2, 5, 6, 3, 5, 6, 3, 2, 2, 5, 5, 3, 1, 4, 2, 3, 1, 6, 6, 2, 2, 5, 5, 1, 1, 5, 3, 5, 3, 6, 2, 5, 6, 6, 5, 6, 5, 6, 6, 4, 2, 6, 6, 3, 1, 3, 5, 1, 6, 2, 2, 3, 6, 5, 1, 1, 6, 3, 5, 2, 6, 1, 2, 6, 4, 3, 4, 5, 2, 3, 1, 3, 4, 6, 5, 6, 3, 1, 4, 4, 6, 5, 2, 5, 2, 6, 1, 2, 6, 5, 3, 6, 5, 6, 1, 6, 3, 6, 6, 6, 6, 1, 2, 6, 6, 5, 6, 2, 1, 2, 1, 5, 1, 3, 5, 4, 1, 2, 3, 5, 6, 2, 2, 1, 2, 4, 2, 4, 4, 4, 2, 3, 6, 6, 5, 4, 6, 6, 6, 4, 6, 1, 6, 6, 3, 1, 4, 3, 2, 4, 5, 2, 5, 3, 5, 2, 4, 6, 5, 6, 2, 3, 5, 4, 2, 6, 6, 6, 6, 6, 1, 3, 5, 6, 5, 2, 4, 2, 5, 4, 6, 1, 5, 4, 1, 4, 6, 3, 1, 6, 2, 2, 3, 6, 4, 3, 6, 3, 3, 3, 3, 5, 5, 1, 3, 6, 1, 1, 1, 3, 6, 2, 2, 3, 5, 5, 2, 6, 6, 6, 1, 5, 1, 1, 6, 1, 5, 5, 4, 5, 2, 1, 1, 1, 6, 3, 6, 6, 6, 6, 3, 6, 4, 1, 6, 2, 1, 2, 3, 3, 4, 6, 4, 6, 6, 3, 5, 4, 1, 4, 2, 6, 2, 1, 3, 6, 6, 4, 2, 5, 6, 3, 4, 1, 6, 1, 5, 6, 5, 3, 6, 6, 6, 6, 6, 6, 5, 6, 6, 4, 3, 3, 6, 4, 4]
['1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'

In [4]:
#Алгоритм Витерби
#Инициализация:
Vk = [[0]*L, [0]*L]
Pv = [[0]*(L - 1), [0]*(L - 1)]
Vk[0][0] = e[0][path[0] - 1]*(2 / 3)
Vk[1][0] = e[1][path[0] - 1]*(1 / 3)
#Рекурсия:
for i in range(1, L):
    Vk[0][i] = e[0][path[i] - 1]*max(Vk[0][i - 1]*a[0][0], Vk[1][i - 1]*a[1][0])
    Vk[1][i] = e[1][path[i] - 1]*max(Vk[1][i - 1]*a[1][1], Vk[0][i - 1]*a[0][1])
    if Vk[0][i - 1]*a[0][0] > Vk[1][i - 1]*a[1][1]:
        Pv[0][i - 1] = 0
    else:
        Pv[0][i - 1] = 1
    if Vk[1][i - 1]*a[1][0] > Vk[0][i - 1]*a[0][1]:
        Pv[1][i - 1] = 1
    else:
        Pv[1][i - 1] = 0

#Нахождение наиболее вероятной траектории:
piv = [0]*L
if Vk[0][L - 1] > Vk[1][L - 1]:
    piv[L - 1] = 0
else:
    piv[L - 1] = 1
for i in range(L - 1):
    piv[L - (i + 2)] = Pv[piv[L - (i + 1)]][L - (i + 2)]
#Наиболее вероятная траектория:
print(piv)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [5]:
#Алоритм просмотра вперед
def forward(e, a, path, L):
    fk = [[0]*(L+1), [0]*(L+1)]
    fk[0][0] = fk[1][0] =1
    fk[0][1] = e[0][path[0]-1]*(2 / 3)
    fk[1][1] = e[1][path[0]-1]*(1 / 3)
    for i in range(2, L+1):
        fk[0][i] = e[0][path[i-1]-1]*(fk[0][i-1]*a[0][0] + fk[1][i-1]*a[1][0])
        fk[1][i] = e[1][path[i-1]-1]*(fk[1][i-1]*a[1][1] + fk[0][i-1]*a[0][1])
    return(fk)


fk = forward(e,a, path, L)
resf = fk[0][-1] + fk[1][-1]

#состояние 0 - честная кость:
print(fk[0])

#состояние 1 - нечестная кость:
print(fk[1])

#проверка
print(resf)

[1, 0.1111111111111111, 0.018148148148148142, 0.0031697530864197515, 0.0005300565843621396, 8.672588305898486e-05, 1.5212570801897565e-05, 2.54917173913561e-06, 4.175328955251277e-07, 7.343284658901361e-08, 1.2320774216332406e-08, 2.019360270378188e-09, 3.5572265748985224e-10, 5.973018944628897e-11, 9.793592731388853e-12, 1.5858978444724715e-12, 2.550887343493216e-13, 4.088014956672153e-14, 6.538146739120455e-15, 1.0445043213530248e-15, 1.667614943317947e-16, 2.6615293206561166e-17, 4.247009976595597e-18, 6.776241570949568e-19, 1.1139148889057727e-19, 1.9764777947541735e-20, 3.330207097689686e-21, 5.470004083360974e-22, 8.86605008528603e-23, 1.4268185260365597e-23, 2.2872422442419354e-24, 3.6586586613156804e-25, 5.84540916292453e-26, 9.332999651065369e-27, 1.4895952819335914e-27, 2.3769866662341584e-28, 3.9087023058948567e-29, 6.339212949140501e-30, 1.0205050557014531e-30, 1.717796583209319e-31, 3.221351736638064e-32, 5.566144986437991e-33, 1.1042807199644824e-33, 1.953759914833082e-34

In [6]:
#Алоритм просмотра назад
def backward(e, a, path, L):
    bk = [[0]*(L+1), [0]*(L+1)]
    bk[0][-1] = bk[1][-1] = 1
    for i in range(L-1,-1,-1):
        bk[0][i] = (e[0][path[i]-1]*bk[0][i+1]*a[0][0] + e[1][path[i]-1]*bk[1][i+1]*a[0][1])
        bk[1][i] = (e[1][path[i]-1]*bk[1][i+1]*a[1][1] + e[0][path[i]-1]*bk[0][i+1]*a[1][0])
    bk[0][0] = bk[0][0]*(2/3)
    bk[1][0] = bk[1][0]*(1/3)
    return(bk)


bk = backward(e,a, path, L)

print(bk[0])
print(bk[1])

#проверка
resb = bk[0][0] + bk[1][0]
print(resb)

[1.152612489624709e-227, 1.0382492277091248e-226, 5.995765983815974e-226, 3.6835412881584844e-225, 2.2247257989018316e-224, 1.2996666676101628e-223, 8.021053782202342e-223, 4.8863177997888384e-222, 2.9035073557223105e-221, 1.8037673337677954e-220, 1.1123634280948574e-219, 6.766601265890121e-219, 4.240920953395247e-218, 2.6576605423821707e-217, 1.6651313366026894e-216, 1.0428816071867113e-215, 6.52723285584219e-215, 4.080338014217172e-214, 2.545134032562672e-213, 1.581231647044798e-212, 9.752408641872302e-212, 5.933763522495225e-211, 3.5174852510523576e-210, 1.977405777142209e-209, 1.201656455514741e-208, 7.528476697278775e-208, 4.714695936302269e-207, 2.9503657961448415e-206, 1.8437970287934732e-205, 1.1494565826063179e-204, 7.134259891850938e-204, 4.3921267567294e-203, 2.66319168519558e-202, 1.5680935963573809e-201, 8.688758205354702e-201, 4.161986056275554e-200, 2.3477633369865767e-199, 1.1776185156996082e-198, 4.070512778829059e-198, 1.833331893771392e-197, 9.998479399925051e-197, 4

In [7]:
#еще одна проверка
for i in range(L+1):
    print(fk[0][i]*bk[0][i] + fk[1][i]*bk[1][i])


1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609122e-227
1.7204377456609124e-227
1.7204377456609122e-227
1.7204377456609124e-227
1.7204377456609126e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609126e-227
1.7204377456609129e-227
1.7204377456609126e-227
1.7204377456609126e-227
1.7204377456609122e-227
1.7204377456609126e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609126e-227
1.7204377456609126e-227
1.7204377456609126e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609126e-227
1.7204377456609124e-227
1.7204377456609122e-227
1.7204377456609124e-227
1.7204377456609126e-227
1.7204377456609129e-227
1.7204377456609129e-227
1.7204377456609126e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.7204377456609124e-227
1.72043774566091

In [8]:
#Апостериорное дешифрование
def aposter(fkk, bkk, L):
    apstr = [0]*(L+1)
    for i in range(L+1):
        apstr[i] = (fkk[0][i]*bkk[0][i])/(fkk[0][-1] + fkk[1][-1])
    return(apstr)


apstr = aposter(fk, bk, L)
print(apstr)

[0.669953035227047, 0.6705329826200556, 0.6324672288221549, 0.6786596258157295, 0.6854247188439991, 0.6551515142881238, 0.7092430334973054, 0.7240054616955601, 0.7046519622525065, 0.7698957444807804, 0.796609972004382, 0.7942284337978227, 0.8768650162151731, 0.9226870782205042, 0.947876097004326, 0.9613272535136315, 0.9677906522333732, 0.9695487600447783, 0.9672224303141872, 0.9599901493465427, 0.9452979292942512, 0.9179573998996304, 0.8683136016646977, 0.7788354599578622, 0.778024674547813, 0.8648884307510674, 0.9126115670341178, 0.9380469007392727, 0.9501765957886859, 0.9532840993949052, 0.9484667868510289, 0.9340234856293074, 0.9048537279845799, 0.8506565857760316, 0.752293028980605, 0.5750272211691702, 0.5333937826067371, 0.43391134394129344, 0.2414489498715718, 0.18305174197392124, 0.18721176665602246, 0.14835760735735543, 0.16217588840898944, 0.13538788151099002, 0.1575770780974313, 0.2559143669295535, 0.29465986551452494, 0.2874960528624503, 0.3570907044021427, 0.588663153244460

In [9]:
#Обучение Баума-Уэлша:
#Введем случаные нормированные матрицы переходов и эмиссий
a_r = np.array([[0.6, 0.4], [0.3, 0.7]], float)
e_r = np.array([[0.11, 0.22, 0.18, 0.19, 0.1, 0.2], [0.15, 0.2, 0.13, 0.17, 0.25, 0.1]], float)
#Рассмотрим некоторое количество последовательноестей:
N = 100
path_bw = np.zeros((N, L), int)
s_bw = np.zeros((N, L), int)
fk_bw = np.zeros((N, 2, L + 1))
bk_bw = np.zeros((N, 2, L + 1))
apstr_bw = np.zeros((N, L + 1))

#Генерируем последовательности
for i in range(N):
    path_bw[i], s_bw[i] = HMM(e, a, L)


In [10]:
#считаем количество переходов, составляем нормированные матрицы Е и А
for j in range(N):
    A00 = int(0)
    A01 = int(0)
    A10 = int(0)
    A11 = int(0)
    E00 = int(0)
    E01 = int(0)
    E02 = int(0)
    E03 = int(0)
    E04 = int(0)
    E05 = int(0)
    E10 = int(0)
    E11 = int(0)
    E12 = int(0)
    E13 = int(0)
    E14 = int(0)
    E15 = int(0)
    fk_bw[j] = forward(e_r, a_r, path_bw[j], L)
    bk_bw[j] = backward(e_r, a_r, path_bw[j], L)
    apstr_bw[j] = aposter(fk_bw[j], bk_bw[j], L)
    print(apstr_bw[j])
    for i in range(L):
        if (apstr_bw[j][i] > 0.5) and (apstr_bw[j][i+1] > 0.5):
            A00 = A00 + 1
        elif (apstr_bw[j][i] > 0.5) and (apstr_bw[j][i+1] <= 0.5):
            A01 = A01 + 1
        elif (apstr_bw[j][i] <= 0.5) and (apstr_bw[j][i+1] > 0.5):
            A10 = A10 + 1
        else:
            A11 = A11 + 1
        if apstr_bw[j][i] > 0.5:    
            if path_bw[j][i] == 1:
                E00 = E00 + 1
            if path_bw[j][i] == 2:
                E01 = E01 + 1
            if path_bw[j][i] == 3:
                E02 = E02 + 1
            if path_bw[j][i] == 4:
                E03 = E03 + 1
            if path_bw[j][i] == 5:
                E04 = E04 + 1
            if path_bw[j][i] == 6:
                E05 = E05 + 1
        else:    
            if path_bw[j][i] == 1:
                E10 = E10 + 1
            if path_bw[j][i] == 2:
                E11 = E11 + 1
            if path_bw[j][i] == 3:
                E12 = E12 + 1
            if path_bw[j][i] == 4:
                E13 = E13 + 1
            if path_bw[j][i] == 5:
                E14 = E14 + 1
            if path_bw[j][i] == 6:
                E15 = E15 + 1
    sumA0 = A00 + A01
    sumA1 = A10 + A11
    print(A00, A01, A10, A11)
    if sumA0 != 0:
        a_r[0][0] = (a_r[0][0] + (A00 / sumA0)) / 2
        a_r[0][1] = (a_r[0][1] + (A01 / sumA0)) / 2
    else:
        a_r[0][0] = a_r[0][0]
        a_r[0][1] = a_r[0][1]
    if sumA1 != 0:
        a_r[1][0] = (a_r[1][0] + (A10 / sumA1)) / 2
        a_r[1][1] = (a_r[1][1] + (A11 / sumA1)) / 2
    else:
        a_r[1][0] = a_r[1][0]
        a_r[1][1] = a_r[1][1] / 2
    sumE0 = E00 + E01 + E02 + E03 + E04 + E05
    sumE1 = E10 + E11 + E12 + E13 + E14 + E15
    if sumE0 != 0:
        e_r[0][0] = (e_r[0][0] + (E00/sumE0)) / 2
        e_r[0][1] = (e_r[0][1] + (E01/sumE0)) / 2
        e_r[0][2] = (e_r[0][2] + (E02/sumE0)) / 2
        e_r[0][3] = (e_r[0][3] + (E03/sumE0)) / 2
        e_r[0][4] = (e_r[0][4] + (E04/sumE0)) / 2
        e_r[0][5] = (e_r[0][5] + (E05/sumE0)) / 2
    else:
        e_r[0][0] = e_r[0][0]
        e_r[0][1] = e_r[0][1]
        e_r[0][2] = e_r[0][2]
        e_r[0][3] = e_r[0][3]
        e_r[0][4] = e_r[0][4]
        e_r[0][5] = e_r[0][5]
    if sumE1 != 0:
        e_r[1][0] = (e_r[1][0] + (E10/sumE1)) / 2
        e_r[1][1] = (e_r[1][1] + (E11/sumE1)) / 2
        e_r[1][2] = (e_r[1][2] + (E12/sumE1)) / 2
        e_r[1][3] = (e_r[1][3] + (E13/sumE1)) / 2
        e_r[1][4] = (e_r[1][4] + (E14/sumE1)) / 2
        e_r[1][5] = (e_r[1][5] + (E15/sumE1)) / 2
    else:
        e_r[1][0] = e_r[1][0]
        e_r[1][1] = e_r[1][1]
        e_r[1][2] = e_r[1][2]
        e_r[1][3] = e_r[1][3]
        e_r[1][4] = e_r[1][4]
        e_r[1][5] = e_r[1][5]
print(a_r, e_r)

[0.65313124 0.73434378 0.67672854 0.51567157 0.42432763 0.55859168
 0.3810066  0.356644   0.43903776 0.46772388 0.4887392  0.53581015
 0.51216652 0.5520174  0.54743219 0.49088576 0.46385634 0.41108338
 0.24799398 0.40706568 0.44930154 0.4423879  0.21425307 0.22617674
 0.51826217 0.42814373 0.25534319 0.42352741 0.34232304 0.3420492
 0.42218357 0.40957791 0.59946279 0.54677313 0.60090427 0.69020196
 0.70526882 0.67648968 0.54582662 0.5206499  0.39969145 0.46195656
 0.50737714 0.59685244 0.44038786 0.53689568 0.28320548 0.44089087
 0.3763287  0.45359543 0.32877941 0.26227539 0.56310193 0.55275623
 0.49909499 0.48806528 0.49005609 0.50825784 0.57184029 0.34405047
 0.20991199 0.29607867 0.37837009 0.29721332 0.21423259 0.36395256
 0.32700378 0.34222928 0.43866823 0.4689742  0.49362228 0.54022439
 0.52325167 0.41484905 0.51567182 0.51303386 0.40260582 0.48029675
 0.57074383 0.64655573 0.56653789 0.46485357 0.65726327 0.70849682
 0.72187521 0.72437818 0.72105339 0.70519586 0.64482622 0.41819

[0.76004587 0.86002409 0.69745373 0.22698995 0.08390037 0.05334211
 0.05306183 0.08579491 0.20147924 0.52083185 0.62630201 0.6472667
 0.60944267 0.46643237 0.45399169 0.3599314  0.35510765 0.42520042
 0.4238096  0.34922909 0.10997331 0.04282466 0.018583   0.01022168
 0.00841882 0.01116442 0.02151952 0.00725452 0.00300217 0.0024884
 0.00261592 0.00237384 0.00260276 0.00366536 0.00798925 0.02188948
 0.07070001 0.07323403 0.03259995 0.02436538 0.03200937 0.0103711
 0.00410981 0.00210182 0.00190515 0.00189326 0.00204459 0.00255644
 0.00263397 0.00236362 0.00254928 0.00402895 0.00689172 0.01807398
 0.0061816  0.0042584  0.00375211 0.00794914 0.02308031 0.01292151
 0.01439137 0.02483937 0.00933083 0.00549944 0.00437385 0.00469917
 0.00941668 0.02175957 0.00948297 0.00483147 0.00628805 0.01123294
 0.02517909 0.01317203 0.01331104 0.02584306 0.0121938  0.0077812
 0.00768573 0.01803008 0.00528276 0.00228329 0.0016241  0.00167722
 0.00149924 0.00173999 0.00186959 0.00328878 0.00743741 0.02168585

0 0 0 300
[5.00818734e-01 3.28140689e-01 7.27479161e-02 3.82023577e-02
 6.14950983e-03 9.77697034e-04 1.43179548e-04 3.04629626e-05
 1.29708456e-05 4.77318738e-06 1.58108025e-07 2.20659515e-06
 1.39142330e-06 3.70347134e-08 2.57736312e-07 3.34584554e-07
 2.94341386e-07 3.07338391e-07 4.45421621e-07 9.64335582e-07
 3.92860592e-07 2.65598550e-08 4.00264837e-07 1.01714977e-06
 6.00516522e-07 1.63175416e-06 2.26703135e-06 4.48296910e-08
 1.38312742e-07 1.09964421e-05 3.32184047e-05 3.78366851e-05
 5.45960621e-05 5.31997509e-05 5.00395883e-05 2.75629832e-05
 3.92969679e-06 7.09755229e-07 1.03666291e-07 5.19625414e-06
 2.72631329e-06 8.17013762e-05 1.63800231e-04 1.95746724e-04
 1.95749084e-04 1.63808655e-04 8.17206658e-05 2.75576471e-06
 5.27950588e-06 2.12804200e-07 3.18989353e-06 2.02505868e-05
 2.31126082e-05 1.34071958e-05 9.78280296e-06 2.27986252e-05
 2.25609506e-05 8.93431428e-06 2.19598414e-06 2.37768794e-06
 1.26369292e-04 1.80728657e-04 1.96438452e-04 1.82452634e-04
 2.57613886e-0

1 1 0 298
[5.46133288e-01 8.45317984e-01 5.86018587e-01 1.10467343e-01
 2.10563468e-02 4.24799575e-03 1.10043435e-03 5.76106924e-04
 8.36354124e-04 4.21205467e-04 1.47996999e-08 1.84695841e-07
 1.00948058e-04 3.31636249e-05 2.14856371e-05 4.11163105e-09
 2.96816220e-07 2.42514820e-04 2.96820189e-07 5.88289056e-09
 8.76507728e-05 5.87435550e-09 1.04724400e-07 4.07371563e-09
 2.02904101e-05 2.29915504e-05 2.33525967e-05 2.34112764e-05
 2.34987743e-05 2.41089249e-05 2.86853871e-05 1.83166890e-07
 5.78357932e-05 3.42101228e-04 1.00334711e-04 6.52393905e-05
 1.13746484e-04 1.94818709e-07 4.19817360e-09 2.48393248e-05
 4.75085800e-09 1.76119298e-05 1.95366288e-07 8.54723669e-05
 5.77153863e-04 4.48648520e-04 2.66762081e-04 3.83305215e-07
 2.82151434e-05 2.09220639e-05 1.37435748e-07 1.76050344e-05
 3.99889303e-09 4.33911601e-09 3.03425674e-05 3.47770270e-05
 1.01642283e-04 3.08053122e-07 8.77724802e-05 8.87314839e-09
 6.74874010e-05 4.24656262e-04 7.58157004e-04 4.38309348e-04
 9.42715216e-0

[1.86085232e+00 5.59573320e-02 3.10550217e-04 1.25286666e-03
 5.01394997e-05 2.12906291e-04 2.13186226e-04 2.15000189e-04
 1.28119175e-03 3.00414943e-03 3.11149876e-03 6.02161922e-03
 1.31111039e-03 2.16854071e-04 1.26403709e-03 1.27102349e-03
 9.23178225e-04 6.13983914e-03 6.15696814e-03 1.31072825e-03
 5.05789398e-05 1.25192495e-03 2.14950409e-04 2.13186613e-04
 2.13183641e-04 2.13199901e-04 2.22762193e-04 5.84464530e-03
 2.32769622e-04 5.84472736e-03 2.24611426e-04 1.25374513e-03
 2.24610664e-04 5.84428327e-03 2.22761577e-04 2.13199899e-04
 2.13183636e-04 2.13183608e-04 2.13183636e-04 2.13199877e-04
 2.22748631e-04 5.83667193e-03 5.30789963e-05 2.90632562e-03
 5.19440806e-05 3.04043226e-03 5.96928229e-03 2.24150488e-04
 8.74515357e-04 2.18953963e-04 2.93009752e-03 9.33689362e-04
 5.92415003e-03 1.30842427e-03 5.05822191e-05 1.26240961e-03
 1.27441229e-03 1.26405202e-03 2.14987441e-04 2.22751719e-04
 5.83661949e-03 5.18939294e-05 5.19438082e-05 5.96236874e-03
 3.06017539e-03 8.929865

[8.25484240e-01 5.59883748e-01 2.19375663e-02 9.41941559e-04
 1.23567738e-04 8.56852627e-06 2.34896764e-06 8.25842566e-07
 1.43182985e-06 7.77411533e-07 1.58499892e-06 8.49735704e-07
 2.56014854e-06 2.93628246e-06 3.23632540e-06 4.46393690e-06
 7.50463966e-07 3.76004169e-06 5.61317753e-07 6.70510373e-07
 7.25487621e-07 1.53376083e-06 5.02266847e-07 1.36196070e-06
 5.03056998e-07 1.55253448e-06 2.14972979e-06 4.31207289e-06
 1.23659750e-06 5.18945664e-06 5.44115336e-06 2.30414534e-06
 1.57394098e-06 5.30558027e-07 1.98369371e-06 4.25494100e-06
 1.18995839e-06 4.63348062e-06 3.40374989e-06 3.61593869e-06
 5.97982980e-06 5.71588485e-06 2.49644114e-06 2.51827398e-06
 1.02770262e-06 4.23094817e-06 2.06072468e-06 2.37962146e-06
 5.41297794e-06 5.00501760e-06 6.49296302e-07 1.38275420e-06
 5.35382328e-07 2.29846818e-06 1.72532703e-06 1.70912049e-06
 2.15156924e-06 4.14582939e-06 6.31188842e-07 1.73486978e-06
 2.17512966e-06 4.29086841e-06 1.13497898e-06 3.87307361e-06
 6.07811292e-07 1.504515

[1.24979071e+00 7.83711467e-02 2.29917688e-03 7.11364314e-04
 1.89325546e-04 8.45858137e-05 5.44242787e-05 2.97760267e-05
 7.65053180e-07 2.00776527e-07 2.21192181e-06 1.82437705e-07
 1.82574467e-07 2.21869627e-06 2.27761697e-07 1.87824356e-06
 2.92993926e-06 6.59680632e-06 2.77336050e-07 1.86829771e-07
 2.30076918e-06 5.52035993e-07 1.36966722e-05 1.85649536e-05
 1.53965817e-05 4.03555410e-06 2.20075066e-07 1.77723854e-07
 1.81767875e-06 2.64109656e-06 2.76151112e-07 3.09160976e-06
 3.61537567e-06 2.56725718e-06 2.85743931e-06 3.77126698e-07
 6.57297064e-06 5.15554440e-06 4.40529014e-06 3.29113794e-06
 2.81915220e-07 2.64602507e-06 1.83107605e-06 2.38371259e-07
 2.68434753e-06 7.55745307e-06 5.13002078e-06 3.45085684e-06
 2.12296429e-07 2.22368522e-07 3.94286411e-06 7.08942549e-06
 3.66808762e-07 2.11483370e-06 4.11145770e-06 5.39841985e-06
 7.74438442e-06 3.79341762e-07 2.00247135e-06 3.45552706e-06
 9.17901329e-06 8.22080063e-06 4.28143325e-07 2.96299234e-06
 2.92848007e-06 2.771765

[7.69569610e-01 6.13528657e-01 1.11094932e-06 1.97954598e-04
 1.37950183e-04 7.45872969e-04 1.90014323e-08 5.01547341e-05
 2.00067355e-04 1.99114523e-04 1.83254502e-08 2.09584028e-04
 7.50748763e-04 1.90097451e-08 5.00223834e-05 1.72819897e-04
 5.09965562e-05 1.97242630e-04 5.03898151e-05 5.02714328e-05
 1.72870271e-04 5.37500844e-05 7.54651172e-04 2.12277962e-04
 1.28734294e-04 1.80562898e-08 1.28410674e-04 1.86095043e-04
 7.58702025e-04 1.38306964e-04 2.12682800e-04 7.65556605e-04
 2.10905480e-04 5.10641200e-05 1.72835465e-04 5.08826548e-05
 1.74167412e-04 1.30451563e-04 1.74169452e-04 5.10033067e-05
 1.97256570e-04 5.11127896e-05 1.96282607e-04 1.80362049e-08
 5.00326871e-05 1.74942153e-04 1.77488170e-04 2.02553218e-04
 2.00145356e-04 5.07905715e-05 1.28331911e-04 1.28327599e-04
 5.04416294e-05 1.29201989e-04 1.98736593e-04 5.03960452e-05
 5.00569212e-05 1.28918272e-04 1.76281394e-04 1.77488052e-04
 2.01203778e-04 1.30325734e-04 1.37213169e-04 7.58529849e-04
 1.84690777e-04 5.008003

 5.66418890e-08]
0 1 0 299
[[1.56886181e-02 9.84311382e-01]
 [2.63164807e-08 9.99999974e-01]] [[0.1260376  0.06274814 0.25398275 0.00201416 0.03174835 0.52346899]
 [0.17874247 0.13734367 0.16380463 0.1380776  0.12911661 0.25291503]]
